In [7]:
from scipy import stats
import numpy as np

### Hypothesis testing with one sample

#### mean - known population standard deviation - use normal

In [39]:
# Left and right tail tests for the mean 
# When the population standard deviation is somehow known, p-values found using a normal distribution

h = 15  # null hypothesis, mu <= h use right tail test, 
        #                  mu >= h use left tail test

n = 10  # number of samples 
mu = 15.2  # Observed average
sigma = 0.5  # Population standard deviation

pval = stats.norm.cdf(mu, h, (sigma / np.sqrt(n))) # Left tail test
pval_lefttail1 = 1 - stats.norm.cdf(mu, h, (sigma / np.sqrt(n))) # Right tail test
pval_lefttail2 = stats.norm.sf(mu, h, (sigma / np.sqrt(n))) # More accurate right tail test

print(pval) # if hypothesis were >=
print(pval_lefttail1) # if hypothesis were >=
print(pval_lefttail2)

0.897048394634
0.102951605366
0.102951605366


#### proportion - treat as binomial modeled as normal

In [40]:
# proportion, two-tailed

h = 0.50  # hypothesis proportion

n = 100   # number of samples
p = 0.53  # proportion observed


sigma = np.sqrt(h * (1 - h) / n)  # for proportion from binomial

# explainatory path to p-value 
if h < p:
    right_tail = stats.norm.sf(p, h, sigma)
    left_tail = stats.norm.cdf(h + (h - p), h, sigma)
else:
    right_tail = stats.norm.sf(h + (h - p), h, sigma)
    left_tail = stats.norm.cdf(p, h, sigma)
pval = right_tail + left_tail
print(pval)

# one line to p-value       
pval = 2 * stats.norm.sf(h + np.abs(h - p), h, sigma)
    
print(pval)

0.5485062355
0.5485062355


#### unknown population standard deviation - use t-test

In [58]:
data = [1.11, 1.07, 1.11, 1.07, 1.12, 1.08, 0.98, 0.98, 1.02, 0.95, 0.95]
h = 1.00 # null hypothesis: mean <= 1

n = len(data)
mu = np.mean(data)
sigma = np.std(data, ddof=1) # sample std so ddof = 1
print(mu)

pval = stats.t.sf(mu, n - 1, h, sigma / np.sqrt(n)) 

print(pval)

# Or just use scipy method
#  Note: returns the test statistic, and 2 sided p-value 
print(stats.ttest_1samp(data, h)) 

# The one-tail that's interesting is half the two-tailed
# * i.e. hypothesis mean <= 1 will never be rejected by a mean <= 1
print(stats.ttest_1samp(data, h)[1] / 2) # 

# If you really want the one tail for the side that will never reject the null  
print(1 - (stats.ttest_1samp(data, h)[1]) / 2) 


1.04
0.0358606486061
Ttest_1sampResult(statistic=2.0137774303955394, pvalue=0.071721297212151217)
0.0358606486061
0.964139351394


### Hypothesis testing with two samples

In [14]:
# Two populations

n_g = 9      # number of girls
mu_g = 2     # average for girls
s_g = 0.866  # sample sigma for girls

n_b = 16     # number of boys
mu_b = 3.2   # average for boys
s_b = 1      # sample sigma for boys

nx = n_g
vx = s_g ** 2
ny = n_b
vy = s_b ** 2

#scipy.stats.ttest_ind(a, b, equal_var=True)
# if equal_var: perform a standard independent 2 sample test that assumes equal population sizes and variances 
# if not equal_var: perform Welch’s t-test

df = ((vx / nx + vy / ny) ** 2 / 
      (1 / (nx - 1) * (vx / nx) ** 2 + 
       1 / (ny - 1) * (vy / ny) ** 2))

print(df)

# dof 18.8462





18.84659125336577


0.784996

In [60]:
# https://gist.github.com/jdmonaco/5922991

def t_welch(x, y, tails=2):
    """Welch's t-test for two unequal-size samples, not assuming equal variances
    """
    assert tails in (1,2), "invalid: tails must be 1 or 2, found %s"%str(tails)
    x, y = np.asarray(x), np.asarray(y)
    nx, ny = x.size, y.size
    vx, vy = x.var(), y.var()


    df = int((vx/nx + vy/ny)**2 / # Welch-Satterthwaite equation
        ((vx/nx)**2 / (nx - 1) + (vy/ny)**2 / (ny - 1)))
    t_obs = (x.mean() - y.mean()) / np.sqrt(vx/nx + vy/ny)
    p_value = tails * stats.t.sf(abs(t_obs), df)
    return TtestResults(t_obs, p_value)